In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('../results/Main/summary.csv')

# data = data.iloc[-21:,:]
total_elapsed_time = data['time_elapsed'].sum()

data = data[['method','test_loss_mean','test_loss_std','dataset_name']]

# Extract noise ratio from dataset name
def extract_noise_ratio(dataset_name):
    parts = dataset_name.split('_')
    for part in parts:
        if 'noise' in part:
            return float(part.split('=')[1])
    return 0.01
        
def extract_core_dataset_name(dataset_name):
    parts = dataset_name.split('_')
    return parts[0]

def dataframe_to_markdown(df):
    # Get the column headers
    headers = df.columns.tolist()
    
    # Create the header row
    header_row = '| | ' + ' | '.join(headers) + ' |'
    
    # Create the separator row
    separator_row = '| --- | ' + ' | '.join(['---'] * len(headers)) + ' |'
    
    # Create the data rows
    data_rows = []
    for index, row in df.iterrows():
        data_row = '| ' + index + ' | ' + ' | '.join(str(cell).strip() for cell in row) + ' |'
        data_rows.append(data_row)
    
    # Combine all rows into a single markdown table string
    markdown_table = '\n'.join([header_row, separator_row] + data_rows)
    
    return markdown_table


data.loc[:,'core_dataset_name'] = data['dataset_name'].apply(extract_core_dataset_name)

data.loc[:,'noise_ratio'] = data['dataset_name'].apply(extract_noise_ratio)

data = data[['method','core_dataset_name','noise_ratio','test_loss_mean','test_loss_std']]
data.columns = ['Method','Dataset','Noise Ratio','Test Loss Mean','Test Loss Std']

# Create a latex table where each row is a method and each column is a dataset
# For each dataset, have a separate subcolumn for each noise ratio
# Each value should show the mean test loss and the standard deviation a a subscript in parentheses

datasets = data['Dataset'].unique()
methods = data['Method'].unique()
noise_ratios = data['Noise Ratio'].unique()

# Construct two-level column names
column_names = []
for dataset in datasets:
    for noise_ratio in noise_ratios:
        column_names.append((dataset,noise_ratio))

latex_table = pd.DataFrame(index=methods,columns=pd.MultiIndex.from_tuples(column_names))

for dataset in datasets:
    for method in methods:
        for noise_ratio in noise_ratios:
            mask = (data['Dataset']==dataset) & (data['Method']==method) & (data['Noise Ratio']==noise_ratio)
            if mask.sum() > 0:
                mean = data.loc[mask,'Test Loss Mean'].values[0]
                std = data.loc[mask,'Test Loss Std'].values[0]
                formatted_noise = f"{std:.3f}".lstrip('0')
                latex_table.loc[method,(dataset,noise_ratio)] = fr"${mean:.3f}_"+"{("+formatted_noise+r")}$"



dataset_dict = {
    'simple':'Logistic Growth',
    'general':'General ODE',
    'tacrolimus':'Pharmacokinetic',
    'mackey':'Mackey-Glass',
    'integral':'Integro-DE',
    'duffing':'Duffing',
    'tumor-real':'Tumor growth (real)',
    'tacrolimus-real':'Drug concentration (real)',
}

method_dict = {
    'NeuralODE':'Neural ODE',
    'NeuralLaplace':'Neural Laplace',
    'DeepONet':'DeepONet',
    'SemanticODE-default':'Semantic ODE',
    'WeakSINDy-5':'WSINDy-5',
    'SINDy-0':'SINDy',
    'WeakSINDy-0':'WSINDy'
    }

In [3]:
latex_table

simple                          tacrolimus  \
                                 0.01              0.20              0.01   
SINDy-5              $0.012_{(.002)}$  $0.222_{(.004)}$  $0.093_{(.004)}$   
WeakSINDy-5          $0.010_{(.000)}$  $0.222_{(.009)}$  $0.211_{(.009)}$   
SINDy-0              $0.012_{(.001)}$  $0.218_{(.011)}$  $0.020_{(.001)}$   
WeakSINDy-0          $0.010_{(.001)}$  $0.217_{(.016)}$  $0.038_{(.006)}$   
PySR-20              $0.012_{(.002)}$  $0.224_{(.007)}$  $0.053_{(.015)}$   
SemanticODE-default  $0.015_{(.005)}$  $0.200_{(.009)}$  $0.023_{(.014)}$   
NeuralODE            $0.023_{(.004)}$  $0.197_{(.005)}$  $0.036_{(.008)}$   
SemanticODE-tumor                 NaN               NaN               NaN   
SemanticODE-pharma                NaN               NaN               NaN   
DeepONet             $0.184_{(.040)}$  $0.306_{(.023)}$  $0.058_{(.010)}$   
NeuralLaplace        $0.126_{(.036)}$  $0.230_{(.017)}$  $0.100_{(.022)}$   

                                                 mackey                    \
                                 0.20              0.01              0.20   
SINDy-5              $0.230_{(.014)}$  $0.238_{(.023)}$  $0.248_{(.025)}$   
WeakSINDy-5          $0.415_{(.299)}$  $0.272_{(.032)}$  $0.300_{(.061)}$   
SINDy-0              $0.209_{(.010)}$  $0.252_{(.026)}$  $0.257_{(.028)}$   
WeakSINDy-0          $0.219_{(.016)}$  $0.200_{(.035)}$  $0.207_{(.031)}$   
PySR-20              $0.242_{(.039)}$  $0.261_{(.021)}$  $0.288_{(.031)}$   
SemanticODE-default  $0.211_{(.015)}$  $0.037_{(.003)}$  $0.075_{(.003)}$   
NeuralODE            $0.203_{(.007)}$  $0.177_{(.010)}$  $0.194_{(.010)}$   
SemanticODE-tumor                 NaN               NaN               NaN   
SemanticODE-pharma                NaN               NaN               NaN   
DeepONet             $0.212_{(.005)}$  $0.107_{(.014)}$  $0.132_{(.012)}$   
NeuralLaplace        $0.229_{(.013)}$  $0.057_{(.006)}$  $0.094_{(.009)}$   

                              general                            integral  \
                                 0.01              0.20              0.01   
SINDy-5              $0.053_{(.012)}$  $0.103_{(.010)}$  $0.431_{(.051)}$   
WeakSINDy-5          $0.066_{(.009)}$  $0.102_{(.008)}$  $0.160_{(.066)}$   
SINDy-0              $0.068_{(.013)}$  $0.115_{(.012)}$  $0.318_{(.172)}$   
WeakSINDy-0          $0.062_{(.009)}$  $0.112_{(.009)}$  $0.152_{(.086)}$   
PySR-20              $0.078_{(.029)}$  $0.119_{(.029)}$  $0.027_{(.011)}$   
SemanticODE-default  $0.015_{(.001)}$  $0.068_{(.002)}$  $0.025_{(.003)}$   
NeuralODE            $0.029_{(.005)}$  $0.075_{(.006)}$  $0.073_{(.007)}$   
SemanticODE-tumor                 NaN               NaN               NaN   
SemanticODE-pharma                NaN               NaN               NaN   
DeepONet             $0.160_{(.033)}$  $0.195_{(.027)}$  $0.100_{(.015)}$   
NeuralLaplace        $0.108_{(.030)}$  $0.138_{(.023)}$  $0.075_{(.044)}$   

                                             tumor-real       \
                                 0.20              0.01 0.20   
SINDy-5              $0.268_{(.019)}$  $0.243_{(.019)}$  NaN   
WeakSINDy-5          $0.452_{(.365)}$  $0.237_{(.015)}$  NaN   
SINDy-0              $0.248_{(.016)}$  $0.249_{(.029)}$  NaN   
WeakSINDy-0          $0.300_{(.082)}$  $0.236_{(.016)}$  NaN   
PySR-20              $0.393_{(.144)}$  $0.536_{(.346)}$  NaN   
SemanticODE-default  $0.203_{(.007)}$  $0.234_{(.019)}$  NaN   
NeuralODE            $0.215_{(.009)}$  $0.228_{(.018)}$  NaN   
SemanticODE-tumor                 NaN  $0.229_{(.019)}$  NaN   
SemanticODE-pharma                NaN               NaN  NaN   
DeepONet             $0.230_{(.014)}$  $0.242_{(.016)}$  NaN   
NeuralLaplace        $0.249_{(.014)}$  $0.243_{(.029)}$  NaN   

                              duffing                     tacrolimus-real       
                                 0.01              0.20              0.01 0.20  
SINDy-5              $0.278_{(

In [4]:
def flatten_noise_level(df):
    df.columns = pd.MultiIndex.from_tuples([(col[0],"low" if col[1]==0.01 else "high") for col in df.columns])
    columns = df.columns.values
    df.columns = [col[0] for col in columns]
    second_level_values = [col[1] for col in columns]
    df.loc['Noise Level'] = second_level_values
    df = df.reindex(['Noise Level'] + [col for col in df.index if col != 'Noise Level'])
    return df


datasets_in_order = ['simple','general','tacrolimus','mackey','integral']
methods_in_order = ['SINDy-5','WeakSINDy-5','PySR-20','SINDy-0','WeakSINDy-0','NeuralODE','NeuralLaplace','DeepONet','SemanticODE-default']

black_box_table = latex_table.loc[methods_in_order,datasets_in_order]
black_box_table = black_box_table.rename(index=method_dict)
black_box_table = black_box_table.rename(columns=dataset_dict)

# Save as a latex table
with open('output/Main Table (Tab 3).tex', 'w') as f:
    f.write(black_box_table.to_latex(escape=False))

black_box_table = flatten_noise_level(black_box_table)

with open('output/Main Table (Tab 3).md', 'w') as f:
    f.write(dataframe_to_markdown(black_box_table))



In [ ]:
methods_in_order = ['SINDy-5','WeakSINDy-5','PySR-20','SINDy-0','WeakSINDy-0','NeuralODE','NeuralLaplace','DeepONet','SemanticODE-default']

duffing_table = latex_table.loc[methods_in_order,['duffing']]

duffing_table = duffing_table.rename(index=method_dict)
duffing_table = duffing_table.rename(columns=dataset_dict)

# Save as a latex table
with open('output/Duffing (Tab 6).tex', 'w') as f:
    f.write(duffing_table.to_latex(escape=False))

duffing_table = flatten_noise_level(duffing_table)

with open('output/Duffing (Tab 6).md', 'w') as f:
    f.write(dataframe_to_markdown(duffing_table))


In [ ]:
real_data_table = methods_in_order = ['SINDy-5','WeakSINDy-5','PySR-20','SINDy-0','WeakSINDy-0','NeuralODE','NeuralLaplace','DeepONet','SemanticODE-default','SemanticODE-tumor','SemanticODE-pharma']

real_data_table = latex_table.loc[methods_in_order,['tumor-real','tacrolimus-real']]

real_data_table = real_data_table.rename(index=method_dict)
real_data_table = real_data_table.rename(columns=dataset_dict)
real_data_table = flatten_noise_level(real_data_table)

real_data_table = real_data_table.iloc[:,[0,2]]

with open('output/Real data (Tab 7).md', 'w') as f:
    f.write(dataframe_to_markdown(real_data_table))

# Save as a latex table
with open('output/Real data (Tab 7).tex', 'w') as f:
    f.write(real_data_table.to_latex(escape=False))
